In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st

### Q1

## part a: get data from PS2

def gen_cohort_category(year1, year2):
    '''
    input:
        year1: int
        year2: int
    output:
        string
    '''
    string_format = "{}-{}"
    
    return string_format.format(year1, year2)

data17_18 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT')
data15_16 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.XPT')
data13_14 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DEMO_H.XPT')
data11_12 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/DEMO_G.XPT')
column_list = ["SEQN", "RIAGENDR", "RIDAGEYR", "RIDRETH3", "DMDEDUC2", "DMDMARTL", "RIDSTATR",
              "SDMVPSU", "SDMVSTRA", "WTMEC2YR", "WTINT2YR"]

data11_12 = data11_12[column_list]
data13_14 = data13_14[column_list]
data15_16 = data15_16[column_list]
data17_18 = data17_18[column_list]

data11_12["cohort"] = gen_cohort_category(11, 12)
data13_14["cohort"] = gen_cohort_category(13, 14)
data15_16["cohort"] = gen_cohort_category(15, 16)
data17_18["cohort"] = gen_cohort_category(17, 18)

dataframe1 = [data11_12, data13_14, data15_16, data17_18]
data1 = pd.concat(dataframe1)

data1.to_pickle("./data1.pkl")
data1 = pd.read_pickle("./data1.pkl")
## part b: merge OHDDESTS to dataframe above

new_column_list = ["SEQN", "RIAGENDR", "RIDAGEYR", "DMDEDUC2", "RIDSTATR"]
data11_12 = data11_12[new_column_list]
data13_14 = data13_14[new_column_list]
data15_16 = data15_16[new_column_list]
data17_18 = data17_18[new_column_list]

new_dataframe1 = [data11_12, data13_14, data15_16, data17_18]
new_data1 = pd.concat(new_dataframe1)

var = ["SEQN", "OHDDESTS"]
url_1718 = "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/OHXDEN_J.XPT"
url_1516 = "https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/OHXDEN_I.XPT"
url_1314 = "https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.XPT"
url_1112 = "https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/OHXDEN_G.XPT"

den_1718 = pd.read_sas(url_1718)
den_1516 = pd.read_sas(url_1516)
den_1314 = pd.read_sas(url_1314)
den_1112 = pd.read_sas(url_1112)

den_1112 = den_1112[var]
den_1314 = den_1314[var]
den_1516 = den_1516[var]
den_1718 = den_1718[var]

new_dataframe2 = [den_1112, den_1314, den_1516, den_1718]
new_data2 = pd.concat(new_dataframe2)
# merge 2 dataframes by the intersection of SEQN
new_df = pd.merge(new_data1, new_data2, on = ["SEQN"], how = "left")
new_col_name = ["id", "gender", "age", "edu", "exam_status", "oral_health_exam_status"]
new_df.columns = new_col_name

new_df["under20"] = np.where(new_df["age"] < 20, 1, 0)
new_df["college"] = np.where(((new_df["edu"] == 4)|(new_df["edu"] == 5)) & (new_df["age"] >= 20), 1, 0)
new_df["ohx"] = np.where((new_df["exam_status"] == 2) & (new_df["oral_health_exam_status"] == 1), 1, 0)
#convert variables to appropriate type and clean the data set
new_df["exam_status"] = pd.Categorical(new_df.exam_status)
new_df["oral_health_exam_status"] = pd.Categorical(new_df.oral_health_exam_status)
new_df["college"] = pd.Categorical(new_df.college)
new_df["ohx"] = pd.Categorical(new_df.ohx)
new_df = new_df.drop(columns = ["edu"])

new_df["gender"] = new_df.gender.replace({1: "Male", 2: "Female"})
new_df["under20"] = new_df.under20.replace({1: "Younger than 20", 0: "Older than 20"})
new_df["college"] = new_df.college.replace({1: "some college/college graduate", 0: "No college/<20"})
new_df["ohx"] = new_df.ohx.replace({1: "complete", 0: "missing"})

new_df # we can see it is a 39156 x 8 df

## part c: drop individuals with exam_status != 2

new_df_remove = new_df[new_df.exam_status == 2]
new_df_remove # we can see it is a 37399 x 8 df
new_df_remove
#Hence, the remaining is 37399, the removed is 39156 - 37399 = 1757

## part d

# continous variable
new_df1 = new_df_remove[["age", "ohx"]]
ohx_vs_age = pd.concat([new_df1.groupby(["ohx"]).mean(), new_df1.groupby(["ohx"]).std()], axis = 1)
ohx_vs_age.columns = ["age_mean", "age_std"]

complete = new_df1.query("ohx == 'complete'")["age"]
missing = new_df1.query("ohx == 'missing'")["age"]
res = st.ttest_ind(complete, missing, equal_var = True)
ohx_vs_age["p-value"] = [res[1], None]
ohx_vs_age

# categorical variable
ohx_vs_gender = new_df_remove.groupby(["gender", "ohx"]).size().unstack()
ohx_vs_under20 = new_df_remove.groupby(["under20", "ohx"]).size().unstack()
ohx_vs_college = new_df_remove.groupby(["college", "ohx"]).size().unstack()

frame = [ohx_vs_gender, ohx_vs_under20, ohx_vs_college]
df = pd.concat(frame, axis = 0)
df["complete ratio"] = (df["complete"]/(df["complete"] + df["missing"])).apply(lambda x: '{:.1%}'.format(x))
df["missing ratio"] = (df["missing"]/(df["complete"] + df["missing"])).apply(lambda x: '{:.1%}'.format(x))

p_gender = st.chi2_contingency(ohx_vs_gender)[1]
p_under20 = st.chi2_contingency(ohx_vs_under20)[1]
p_college = st.chi2_contingency(ohx_vs_college)[1]

df["p-value"] = [p_gender, None, p_under20, None, p_college, None] # each 2x2 df has 1 p-value
df